In [1]:
# Import Dependencies
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True

## Sequential API

In [4]:
def build_model_with_sequential():
    
    # instantiate a Sequential class and linearly stack the layers of your model
    seq_model = nn.Sequential(
        nn.Linear(in_features=784, out_features=128),
        nn.ReLU(),
        nn.Linear(in_features=128, out_features=10),
        nn.LogSoftmax(dim=1)
    )
    
    return seq_model

## Functional API

In [5]:
class Net(nn.Module):
    def __init__(self, in_features, hidden_dim, num_classes):
        super(Net, self).__init__()
        
        self.in_features = in_features
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        
        self.layer1 = nn.Linear(in_features=self.in_features, out_features=self.hidden_dim)
        self.layer2 = nn.Linear(in_features=self.hidden_dim, out_features=self.num_classes)
    
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        x = F.log_softmax(x, dim=1)
        
        return x

In [6]:
def build_model_with_functional():
    
    func_model = Net(in_features=784, hidden_dim=128, num_classes=10)
    
    return func_model

## Build Model and Visualize Model

In [7]:
sequential_model = build_model_with_sequential()
sequential_model.to(device)
print(sequential_model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)


In [8]:
functional_model = build_model_with_functional()
functional_model.to(device)
print(functional_model)

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=10, bias=True)
)


## Training the Model

In [9]:
# Preparing Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

train = datasets.FashionMNIST('./', download=True, train=True, transform=transform)
val = datasets.FashionMNIST('./', download=True, train=False, transform=transform)

trainloader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(val, batch_size=64, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
# Helper Class to track average loss
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [11]:
def train_model(model):
    model.train()
    losses = AverageMeter()
    
    running_loss = 0
    for idx, (images, labels) in enumerate(trainloader):
        images = images.view(images.shape[0], -1)
        images = images.to(device)
        labels = labels.to(device)

        batch_size = images.size(0)

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        losses.update(loss.item(), batch_size)
    
    return losses.avg

In [12]:
def eval_model(model):
    model.eval()
    losses = AverageMeter()
    correct = 0
    total = 0
    
    with torch.no_grad():
      for idx, (images, labels) in enumerate(valloader):
          images = images.view(images.shape[0], -1)
          images = images.to(device)
          labels = labels.to(device)
          
          batch_size = images.size(0)
          
          outputs = model(images)
          loss = criterion(outputs, labels)
          losses.update(loss.item(), batch_size)
          
          _, predictions = torch.max(outputs, 1)
          correct += (predictions == labels).sum().item()
    
    return losses.avg, 100. * correct / len(valloader.dataset)

In [13]:
# Train the Model
EPOCHS = 5

# Model Loss and Optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(sequential_model.parameters(), lr = 0.001)

for epoch in tqdm(range(EPOCHS)):
    train_loss = train_model(sequential_model)
    val_loss, val_acc = eval_model(sequential_model)
    print(f"EPOCH: {epoch}, train_loss: {train_loss}, val_loss: {val_loss}, val_acc: {val_acc}\n")

 20%|██        | 1/5 [00:09<00:36,  9.24s/it]

EPOCH: 0, train_loss: 0.5024896764278411, val_loss: 0.43930438976287844, val_acc: 84.08




 40%|████      | 2/5 [00:18<00:27,  9.15s/it]

EPOCH: 1, train_loss: 0.38669134346644085, val_loss: 0.39745709319114686, val_acc: 85.69




 60%|██████    | 3/5 [00:27<00:18,  9.13s/it]

EPOCH: 2, train_loss: 0.34543906332651775, val_loss: 0.37159491224288943, val_acc: 86.89




 80%|████████  | 4/5 [00:36<00:09,  9.17s/it]

EPOCH: 3, train_loss: 0.3209752576669057, val_loss: 0.3585161651611328, val_acc: 87.39



100%|██████████| 5/5 [00:46<00:00,  9.33s/it]

EPOCH: 4, train_loss: 0.3056336935043335, val_loss: 0.3635697220802307, val_acc: 87.13



In [14]:
# Train the Model
EPOCHS = 5

# Model Loss and Optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(functional_model.parameters(), lr = 0.001)

for epoch in tqdm(range(EPOCHS)):
    train_loss = train_model(functional_model)
    val_loss, val_acc = eval_model(functional_model)
    print(f"EPOCH: {epoch}, train_loss: {train_loss}, val_loss: {val_loss}, val_acc: {val_acc}\n")

 20%|██        | 1/5 [00:10<00:41, 10.44s/it]

EPOCH: 0, train_loss: 0.5007131893952688, val_loss: 0.45968274450302127, val_acc: 83.28




 40%|████      | 2/5 [00:20<00:30, 10.31s/it]

EPOCH: 1, train_loss: 0.37820769069194793, val_loss: 0.3899271595001221, val_acc: 85.82




 60%|██████    | 3/5 [00:29<00:19,  9.95s/it]

EPOCH: 2, train_loss: 0.3426969712257385, val_loss: 0.3720832130432129, val_acc: 86.64




 80%|████████  | 4/5 [00:38<00:09,  9.68s/it]

EPOCH: 3, train_loss: 0.3193135728120804, val_loss: 0.373526788520813, val_acc: 86.36



100%|██████████| 5/5 [00:47<00:00,  9.54s/it]

EPOCH: 4, train_loss: 0.3004917617082596, val_loss: 0.35398429894447325, val_acc: 87.58

